In [1]:
!pip install folium
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import folium

In [2]:
df = pd.read_csv("/content/drive/MyDrive/dados_colab/ponta_negra/asc_202308201757.csv", sep=";")

Separar os valores e adicionar as novas colunas

In [3]:
new_columns = df['valores'].str.split(';', expand=True)
new_column_names = [
    'contador',
    'hora',
    'minuto',
    'segundo',
    'dia',
    'mes',
    'ano',
    'latitude',
    'longitude',
    'temperatura',
    'umidade',
    'tamanho',
    'rssi']
new_columns.columns = new_column_names

# Concatenar as novas colunas no DataFrame original
df = pd.concat([df, new_columns], axis=1)

# Removendo colunas desnecessárias
df.drop(columns = ['valores'], inplace=True)
# df.head()

Criando o DataFrame do Mock

In [4]:
df_mock = pd.DataFrame(df[
      (df['dia'] == '4') &
      (df['mes'] == '5') &
      (df['ano'] == '6')
])
print(f'Qtd total {df.count()[0]}')
print(f'Qtd mock {df_mock.count()[0]}')
print(f'Qtd barco {df.count()[0] - df_mock.count()[0]}')

Qtd total 7793
Qtd mock 6156
Qtd barco 1637


Criando o DataFrame do barco

In [5]:
df_barco = pd.DataFrame(df[
      (df['dia'] != '4') |
      (df['mes'] != '5') |
      (df['ano'] != '6')
])

print(f'Qtd barco {df_barco.count()[0]}')

Qtd barco 1637


Formatando a data hora do GPS

In [6]:
def create_datetime(row):
    ano = str(row['ano']).zfill(4)
    mes = str(row['mes']).zfill(2)
    dia = str(row['dia']).zfill(2)
    hora = str(row['hora']).zfill(2)
    minuto = str(row['minuto']).zfill(2)
    segundo = str(row['segundo']).zfill(2)
    return f'{ano}-{mes}-{dia} {hora}:{minuto}:{segundo}'

df['data_hora_gps'] = df.apply(create_datetime, axis=1)
df.drop(columns = ['hora', 'minuto', 'segundo', 'dia', 'mes', 'ano'], inplace=True)

df_mock['data_hora_gps'] = df_mock.apply(create_datetime, axis=1)
df_mock.drop(columns = ['hora', 'minuto', 'segundo', 'dia', 'mes', 'ano'], inplace=True)

df_barco['data_hora_gps'] = df_barco.apply(create_datetime, axis=1)
df_barco.drop(columns = ['hora', 'minuto', 'segundo', 'dia', 'mes', 'ano'], inplace=True)

Exporta os dados para um arquivo em csv

In [7]:
df.to_csv('todos.csv', index=False, sep=';')
df_mock.to_csv('mock.csv', index=False, sep=';')
df_barco.to_csv('barco.csv', index=False, sep=';')

Converte as colunas para os tipos int32, float64 e datetime

In [8]:
df_barco.reset_index(drop=True, inplace=True)
df_barco['id'] = df_barco['id'].astype(np.int32)
df_barco['data_hora_criacao'] = df_barco['data_hora_criacao'].astype(np.datetime64)
df_barco['data_hora_criacao'] = df_barco['data_hora_criacao'] - np.timedelta64(4, 'h')
df_barco['contador'] = df_barco['contador'].astype(np.int32)
df_barco['latitude'] = df_barco['latitude'].astype(np.float64)
df_barco['longitude'] = df_barco['longitude'].astype(np.float64)
df_barco['rssi'] = df_barco['rssi'].astype(np.int32)
# print(type(df_barco))
print(df_barco.dtypes)
# print(df_barco.columns.values)

id                            int32
data_hora_criacao    datetime64[ns]
contador                      int32
latitude                    float64
longitude                   float64
temperatura                  object
umidade                      object
tamanho                      object
rssi                          int32
data_hora_gps                object
dtype: object


In [9]:
df_barco.head()

,id,data_hora_criacao,contador,latitude,longitude,temperatura,umidade,tamanho,rssi,data_hora_gps
0,108,2023-08-06 11:03:53.965,3358024,-3.049985,-60.109554,-0.0,-0.0,52,-104,2023-08-255 15:03:6365216
1,109,2023-08-06 11:03:54.478,3634,-3.049781,-60.109680,42.7,44.3,52,-103,2023-08-06 15:03:38
2,110,2023-08-06 11:03:54.485,268469815,-3.049607,-60.109825,42.6,44.5,52,-103,2023--977587640-06 858972101:12288:44
3,111,2023-08-06 11:03:54.492,1287690148,-0.000000,873991.500000,46.4,-11378.7,52,-98,788267-96505864-8388674 1711498057:-1869551383...
4,112,2023-08-06 11:03:54.501,3641,-3.049551,-60.109875,42.6,44.7,52,-102,2023-08-06 15:03:46


# Criação do mapa

### Vermelho: horário da manhã
### Verde: horário da tarde

In [10]:
m = folium.Map(location=[-3.084228, -60.104280], zoom_start=13)
folium.Marker(location=[-3.063791, -60.102799]).add_to(m)

# Adicione marcadores para cada coordenada na lista
for lat, lon, dt in zip(df_barco.latitude.values, df_barco.longitude.values, df_barco.data_hora_criacao.values):
  if not np.isnan(lat) and not np.isnan(lon):
    data_comparacao = pd.to_datetime('2023-08-06 15:00:00')
    if data_comparacao > dt:
      folium.CircleMarker(
          location=[lat, lon],
          radius=1,  # Tamanho do círculo
          color='green',  # Cor da borda do círculo
          fill=True,
          fill_color='red',  # Cor de preenchimento do círculo
          fill_opacity=0,  # Opacidade do preenchimento
      ).add_to(m)
    else:
      folium.CircleMarker(
          location=[lat, lon],
          radius=1,  # Tamanho do círculo
          color='red',  # Cor da borda do círculo
          fill=True,
          fill_color='red',  # Cor de preenchimento do círculo
          fill_opacity=0,  # Opacidade do preenchimento
      ).add_to(m)

# Exiba o mapa
m

In [11]:
m.save('map.html')